In [2]:
import os
import cv2
import glob
import h5py
import shutil
import imgaug as aug
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential, Model, load_model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,Activation
from keras.models import Sequential
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
import PIL
import scipy as sp 
import scipy.ndimage as spi
%matplotlib inline

color = sns.color_palette()
%matplotlib inline
%config InlineBackend.figure_format="svg"

import tensorflow as tf

Using TensorFlow backend.


In [3]:
# Set the seed for hash based operations in python
os.environ['PYTHONHASHSEED'] = '0'

seed=1234

# Set the numpy seed
np.random.seed(seed)

# Set the random seed in tensorflow at graph level
tf.set_random_seed(seed)

# Make the augmentation sequence deterministic
aug.seed(seed)

In [4]:
training_data = Path('training') 
validation_data = Path('validation') 
labels_path = Path('monkey_labels.txt')

In [5]:
monkey_labels = []

# Read the file
lines = labels_path.read_text().strip().splitlines()[1:]

In [6]:
lines

['n0   , alouatta_palliata\t , mantled_howler                , 131          , 26',
 'n1   , erythrocebus_patas\t , patas_monkey                  , 139          , 28',
 'n2   , cacajao_calvus\t     , bald_uakari                   , 137          , 27',
 'n3   , macaca_fuscata\t     , japanese_macaque              , 152          , 30',
 'n4   , cebuella_pygmea\t     , pygmy_marmoset                , 131          , 26',
 'n5   , cebus_capucinus\t     , white_headed_capuchin         , 141          , 28',
 'n6   , mico_argentatus\t     , silvery_marmoset              , 132          , 26',
 'n7   , saimiri_sciureus\t     , common_squirrel_monkey        , 142          , 28',
 'n8   , aotus_nigriceps\t     , black_headed_night_monkey     , 133          , 27',
 'n9   , trachypithecus_johnii , nilgiri_langur                , 132          , 26']

In [7]:
for line in lines:
    line = line.split(',')
    line = [x.strip(' \n\t\r') for x in line]
    line[3], line[4] = int(line[3]), int(line[4])
    line = tuple(line)
    monkey_labels.append(line)

In [8]:
monkey_labels = pd.DataFrame(monkey_labels, columns=['Label', 'Latin Name', 'Common Name','Train Images', 'Validation Images'], index=None)


In [9]:
monkey_labels

,Label,Latin Name,Common Name,Train Images,Validation Images
0,n0,alouatta_palliata,mantled_howler,131,26
1,n1,erythrocebus_patas,patas_monkey,139,28
2,n2,cacajao_calvus,bald_uakari,137,27
3,n3,macaca_fuscata,japanese_macaque,152,30
4,n4,cebuella_pygmea,pygmy_marmoset,131,26
5,n5,cebus_capucinus,white_headed_capuchin,141,28
6,n6,mico_argentatus,silvery_marmoset,132,26
7,n7,saimiri_sciureus,common_squirrel_monkey,142,28
8,n8,aotus_nigriceps,black_headed_night_monkey,133,27
9,n9,trachypithecus_johnii,nilgiri_langur,132,26


In [10]:
labels=pd.DataFrame()
labels["id"] = monkey_labels["Label"].str.strip()
labels["name"] = monkey_labels["Common Name"].str.strip()
labels

,id,name
0,n0,mantled_howler
1,n1,patas_monkey
2,n2,bald_uakari
3,n3,japanese_macaque
4,n4,pygmy_marmoset
5,n5,white_headed_capuchin
6,n6,silvery_marmoset
7,n7,common_squirrel_monkey
8,n8,black_headed_night_monkey
9,n9,nilgiri_langur


In [11]:
# Create a dictionary to map the labels to integers
m_id= labels["id"]
m_id

0    n0
1    n1
2    n2
3    n3
4    n4
5    n5
6    n6
7    n7
8    n8
9    n9
Name: id, dtype: object

In [12]:
entries = os.listdir(training_data)
for entry in entries:
    for f in glob.glob(os.path.join(os.path.join(training_data,entry),"*.jpg")):
        W = 250.
        oriimg = cv2.imread(f,cv2.IMREAD_COLOR)
        depth = oriimg.shape
        imgScale = W/250
        newX,newY = 250*imgScale, 250*imgScale
        newimg = cv2.resize(oriimg,(int(newX),int(newY)))
        cv2.imwrite(f,newimg)        

entries2 = os.listdir(validation_data)

for entry in entries2:
    for f in glob.glob(os.path.join(os.path.join(validation_data,entry),"*.jpg")):
        W = 250.
        oriimg = cv2.imread(f,cv2.IMREAD_COLOR)
        depth = oriimg.shape
        imgScale = W/250
        newX,newY = 250*imgScale, 250*imgScale
        newimg = cv2.resize(oriimg,(int(newX),int(newY)))
        cv2.imwrite(f,newimg)
        
        

In [13]:
# This function prepares a random batch from the dataset
def load_batch(dataset_df, batch_size = 25):
    batch_df = dataset_df.loc[np.random.permutation(np.arange(0,
                                                              len(dataset_df)))[:batch_size],:]
    return batch_df

In [16]:
image_width=250
image_height=250
batch_size= 16


In [17]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [18]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
train_generator = train_datagen.flow_from_directory(training_data, 
                                                    target_size=(image_width, image_height), 
                                                    batch_size = batch_size, 
                                                    shuffle=True, # By shuffling the images we add some randomness and prevent overfitting
                                                    class_mode="categorical")

Found 1096 images belonging to 10 classes.


In [20]:
validation_generator = validation_datagen.flow_from_directory(validation_data, 
                                                    target_size=(image_width, image_height), 
                                                    batch_size = batch_size, 
                                                    shuffle=True,
                                                    class_mode="categorical")

Found 272 images belonging to 10 classes.


In [21]:
training_samples = 1097
validation_samples = 272
total_steps = training_samples // batch_size

In [22]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3), pooling="max")

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
for layer in model.layers[:-5]:
        layer.trainable = False

In [25]:
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7ff3d325e1d0> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d2ce5e80> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d28727f0> False
<keras.layers.pooling.MaxPooling2D object at 0x7ff3d2967438> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d338bef0> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d2e466d8> False
<keras.layers.pooling.MaxPooling2D object at 0x7ff3d2807b70> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d28168d0> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d33b82b0> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d289fe48> False
<keras.layers.pooling.MaxPooling2D object at 0x7ff3d28816d8> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d28819b0> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d26b54a8> False
<keras.layers.convolutional.Conv2D object at 0x7ff3d26cdac8> False
<keras.layers.pooling.MaxPooling2D object at 0x7ff3d2679240>

In [29]:
transfer_model = Sequential()
for layer in model.layers:
    transfer_model.add(layer)
transfer_model.add(Dense(512, activation="relu"))
transfer_model.add(Dropout(0.5))
transfer_model.add(Dense(10, activation="softmax"))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [30]:
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 62, 62, 256)       295168    
__________

In [31]:
from keras import optimizers
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.00001)

transfer_model.compile(loss="categorical_crossentropy",
                      optimizer=adam,
                      metrics=["accuracy"])

In [32]:
model_history = transfer_model.fit_generator(train_generator, steps_per_epoch=training_samples // batch_size,
                                            epochs=25,
                                            validation_data=validation_generator,
                                            validation_steps=validation_samples // batch_size)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/25
68/68 [==============================] - 880s 13s/step - loss: 2.0709 - acc: 0.2730 - val_loss: 1.2820 - val_acc: 0.6066
Epoch 2/25
68/68 [==============================] - 765s 11s/step - loss: 1.2319 - acc: 0.5698 - val_loss: 0.7529 - val_acc: 0.7831
Epoch 3/25
68/68 [==============================] - 739s 11s/step - loss: 0.7813 - acc: 0.7418 - val_loss: 0.5584 - val_acc: 0.8051
Epoch 4/25
68/68 [==============================] - 721s 11s/step - loss: 0.5360 - acc: 0.8226 - val_loss: 0.3727 - val_acc: 0.9007
Epoch 5/25
68/68 [==============================] - 715s 11s/step - loss: 0.4346 - acc: 0.8649 - val_loss: 0.3069 - val_acc: 0.9007
Epoch 6/25
68/68 [==============================] - 726s 11s/step - loss: 0.3367 - acc: 0.8787 - val_loss: 0.2672 - val_acc: 0.9301
Epoch 7/25
68/68 [==============================] - 724s 11s/step - loss: 0.3172 -